# Advanced Semantic Kernel - Advanced Agent Features

Welcome to the advanced agent features tutorial! This notebook builds on the basics to explore sophisticated individual agent capabilities and enhancements.

## Prerequisites

Before starting this tutorial, make sure you've completed the **Semantic Kernel Basics** notebook. You should be familiar with:
- Basic agent creation and configuration
- Chat completion services
- Plugins and function calling
- Conversation threads

## What You'll Learn

- **Streaming Responses**: Real-time response generation with `invoke_stream()`
- **Intermediate Steps**: Understanding agent processing with `invoke()`
- **Structured Outputs**: Enforcing consistent response formats with Pydantic
- **Reasoning Models**: Enhanced reasoning capabilities with specialized models

## Resources
- [Agent API Reference](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-api?pivots=programming-language-python)
- [Reasoning Models](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/reasoning?tabs=python-secure%2Cpy)

## Next Steps
After completing this tutorial, continue with the **Semantic Kernel Multi-Agent Orchestration** notebook to learn about agent collaboration patterns.

Let's explore these advanced agent features!

## Setup and Imports

Let's start by importing the necessary modules for advanced agent features.

In [1]:
import asyncio
import json
import os
from dotenv import load_dotenv
from pydantic import BaseModel

try:
    from typing import Annotated
except ImportError:
    from typing_extensions import Annotated

# Core Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.agents import Agent, ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings, AzureChatPromptExecutionSettings
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.contents import ChatMessageContent, TextContent, StreamingChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent

# Load environment variables
load_dotenv()

print("✅ Advanced agent imports loaded successfully!")

✅ Advanced agent imports loaded successfully!


In [2]:
# Configure the main chat completion service
chat_completion = AzureChatCompletion(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
)

print("✅ Chat completion services configured!")

✅ Chat completion services configured!


## 1. Advanced Agent Features

Let's explore advanced agent capabilities including streaming responses and intermediate step handling.

### 1.1 Streaming Responses

Streaming allows you to see responses as they're generated, providing a better user experience for long responses.

In [3]:
# Create a simple agent for streaming demonstration
streaming_agent = ChatCompletionAgent(
    service=chat_completion,
    name="StreamingExpert",
    instructions="You are an expert who provides detailed explanations. Give comprehensive responses about topics.",
)

In [4]:
# Example 1: Regular (non-streaming) response
print("📝 Regular Response:")
print("=" * 40)
response = await streaming_agent.get_response(
    messages="Explain the benefits of renewable energy in detail."
)
print(response.message.content)

print("\n" + "=" * 40)

📝 Regular Response:
Renewable energy offers a wide array of benefits—environmental, economic, social, and health-related. Here’s a comprehensive overview:

**1. Environmental Benefits**

- **Reduced Greenhouse Gas Emissions:** Renewable sources like solar, wind, hydro, geothermal, and biomass generate minimal or zero direct greenhouse gas emissions compared to fossil fuels, which helps combat global warming and climate change.
- **Lower Air and Water Pollution:** Renewable energy production typically emits little to no air pollutants (sulfur dioxide, nitrogen oxides) and substantially less particulate matter. Hydroelectric and wind, for instance, do not produce harmful byproducts as coal or gas plants do. This also means less acid rain and smog.
- **Decreased Water Consumption:** Solar and wind energy require little to no water for operation, unlike coal or nuclear plants, which consume large amounts for cooling. This conserves freshwater resources.
- **Biodiversity Protection:** Shift

In [5]:
# Example 2: Streaming response using invoke_stream()
print("🌊 Streaming Response:")
print("=" * 40)
async for streaming_response in streaming_agent.invoke_stream(
    messages="Explain the challenges of implementing AI in healthcare."
):
    # This shows the response as it's being generated in real-time
    print(streaming_response.content, end="", flush=True)

print("\n\n✅ Streaming examples completed!")

🌊 Streaming Response:
Implementing AI in healthcare offers transformative potential but comes with significant challenges. These challenges span technical, ethical, regulatory, organizational, and social dimensions:

**1. Data Quality and Availability**  
- *Fragmented Data*: Healthcare data often resides in disparate systems that lack interoperability. Electronic health records (EHRs), lab results, imaging, and physician notes are seldom standardized, hindering data aggregation.
- *Data Quality*: Medical data is prone to errors, missing values, and inconsistencies. Inaccurate input can lead to unreliable AI outputs.
- *Limited Labeled Data*: High-quality labeled data, especially for complex diagnostics, requires expert annotation, which is labor-intensive and costly.

**2. Privacy and Security Concerns**  
- *Sensitive Information*: Health data is highly sensitive, and improper handling can result in privacy breaches.
- *Compliance*: AI systems must comply with regulations (e.g., HIPA

### 1.2 Intermediate Steps with invoke()

**Understanding Agent Methods:**

- **`get_response()`**: Provides the final response only - blocks until the agent is completely done processing
- **`invoke()`**: Allows you to see the agent's step-by-step thinking process, including intermediate function calls and results
- **`invoke_stream()`**: Provides real-time streaming of the response as it's being generated

The `invoke()` method is particularly useful when you want to understand how the agent is working with plugins and function calls.

In [6]:
class AnalysisPlugin:
    """Plugin that provides analysis capabilities."""
    
    @kernel_function(description="Analyze a topic and provide insights.")
    def analyze_topic(self, topic: Annotated[str, "The topic to analyze"]) -> str:
        # Simulate analysis processing
        return f"Analysis of '{topic}': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility."
    
    @kernel_function(description="Generate recommendations based on analysis.")
    def generate_recommendations(self, analysis: Annotated[str, "The analysis to base recommendations on"]) -> str:
        return "Recommendations: 1) Conduct further research, 2) Develop a prototype, 3) Test with users, 4) Iterate based on feedback."

# Create an agent with the analysis plugin for the next section
analysis_agent = ChatCompletionAgent(
    service=chat_completion,
    name="AnalysisExpert",
    instructions="You are an expert analyst. Use your tools to provide thorough analysis and recommendations.",
    plugins=[AnalysisPlugin()]
)

In [7]:
async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    """Handle intermediate steps in the agent's processing."""
    for item in message.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"🔧 Function Result: {item.result}")
        elif isinstance(item, FunctionCallContent):
            print(f"📞 Function Call: {item.name} with arguments: {item.arguments}")
        else:
            print(f"💭 {message.name}: {message.content}")

# Test intermediate steps
task = "Analyze the potential of AI-powered educational tools and provide recommendations."
print(f"🎯 Task: {task}")
print("=" * 60)

async for response in analysis_agent.invoke(
    messages=task,
    on_intermediate_message=handle_intermediate_steps
):
    print(f"✅ Final Response from {response.name}: {response.content}")

🎯 Task: Analyze the potential of AI-powered educational tools and provide recommendations.
📞 Function Call: AnalysisPlugin-analyze_topic with arguments: {"topic":"potential of AI-powered educational tools"}
🔧 Function Result: Analysis of 'potential of AI-powered educational tools': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility.
📞 Function Call: AnalysisPlugin-generate_recommendations with arguments: {"analysis":"AI-powered educational tools have significant potential to transform the learning landscape. They leverage artificial intelligence to provide personalized learning experiences, automate administrative tasks, and enable data-driven insights for both educators and students. Key opportunities include adaptive learning platforms, intelligent tutoring systems, AI-driven content creation, real-time performance tracking, and accessibility improvements for students with special needs. The m

In [14]:
# Straming with intermediate steps
async for streaming_response in analysis_agent.invoke_stream(
    messages=task,
    on_intermediate_message=handle_intermediate_steps
):
    print(streaming_response.content, end="", flush=True)
    

📞 Function Call: AnalysisPlugin-analyze_topic with arguments: {"topic":"AI-powered educational tools"}
🔧 Function Result: Analysis of 'AI-powered educational tools': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility.
📞 Function Call: AnalysisPlugin-generate_recommendations with arguments: {"analysis":"AI-powered educational tools have significantly evolved, leveraging machine learning, natural language processing, and adaptive learning technologies. They offer personalized learning paths, instant feedback, and can accommodate diverse student needs at scale. Key benefits include improved engagement, scalable tutoring, data-driven insights for educators, and support for lifelong learning. Challenges include potential bias in algorithms, data privacy concerns, digital divide issues, and the need for teacher training and curriculum integration. Market trends indicate growing adoption in K-12, highe

### 1.3 Structured Outputs with Pydantic

You can enforce structured responses using Pydantic models, ensuring consistent output format.

In [9]:
# Define structured output models
class Step(BaseModel):
    explanation: str
    output: str

class MathSolution(BaseModel):
    steps: list[Step]
    final_answer: str

# Create an agent with structured output
settings = AzureChatPromptExecutionSettings()
settings.response_format = MathSolution

math_agent = ChatCompletionAgent(
    service=chat_completion,
    name="MathTutor",
    instructions="You are a math tutor. Solve problems step by step and provide clear explanations.",
    arguments=KernelArguments(settings=settings)
)

# Test structured output
problem = "Solve: 3x + 7 = 22"
print(f"🧮 Problem: {problem}")

response = await math_agent.get_response(messages=problem)
solution = MathSolution.model_validate(json.loads(response.message.content))

print("\n📚 Structured Solution:")
print(solution.model_dump_json(indent=2))

🧮 Problem: Solve: 3x + 7 = 22

📚 Structured Solution:
{
  "steps": [
    {
      "explanation": "First, subtract 7 from both sides to isolate the term with x.",
      "output": "3x + 7 - 7 = 22 - 7 \n3x = 15"
    },
    {
      "explanation": "Now, divide both sides by 3 to solve for x.",
      "output": "3x / 3 = 15 / 3 \nx = 5"
    }
  ],
  "final_answer": "x = 5"
}


### 1.4 Reasoning Models (if available)

Some models like O1 provide enhanced reasoning capabilities. Let's test if we have access to reasoning models.

In [10]:
# Configure reasoning model (if available)
reasoning_completion = None
if os.getenv("AZURE_REASONING_ENDPOINT"):
    reasoning_completion = AzureChatCompletion(
        api_key=os.getenv("AZURE_REASONING_API_KEY"),
        endpoint=os.getenv("AZURE_REASONING_ENDPOINT"),
        deployment_name=os.getenv("AZURE_REASONING_DEPLOYMENT_NAME"),
        # Reasoning models from OpenAI use "developer" role instead of "system"
        instruction_role="developer",
        service_id="reasoning"
    )
    
    reasoning_settings = AzureChatPromptExecutionSettings(
        service_id="reasoning",
        reasoning_effort="high" # low | medium | high
    )
    
    reasoning_agent = ChatCompletionAgent(
        name="ReasoningExpert",
        instructions="You are an expert problem solver. Think through complex problems step by step.",
        service=reasoning_completion,
    )
    
    print("✅ Reasoning model available!")


✅ Reasoning model available!


In [11]:
if reasoning_completion:
    print("🧠 Testing reasoning model...")
    
    thread = ChatHistoryAgentThread()
    response = await reasoning_agent.get_response(
        messages="Solve: 2x + 3y = 7, 4x - y = 1. Show your reasoning.", 
        thread=thread
    )
    
    print(f"🧠 Reasoning Response: {response.message.content}")

    # Add another user message to the thread
    print("\n🤔 User: Is the solution correct? Explain.")
    response = await reasoning_agent.get_response(
        messages="Is the solution correct? Explain.", 
        thread=thread
    )
    print(f"🧠 Reasoning Response: {response.message.content}")
else:
    print("⚠️ Reasoning model not configured. Skipping reasoning example.")
    print("💡 To enable reasoning, configure AZURE_REASONING_* environment variables.")

🧠 Testing reasoning model...
🧠 Reasoning Response: Here’s one way to solve the system step by step:

1. Write down the equations:  
   (1) 2x + 3y = 7  
   (2) 4x − y = 1  

2. Solve equation (2) for y:  
   4x − y = 1  
   ⇒ −y = 1 − 4x  
   ⇒ y = 4x − 1  

3. Substitute y = 4x − 1 into equation (1):  
   2x + 3(4x − 1) = 7  
   2x + 12x − 3 = 7  
   14x − 3 = 7  
   14x = 10  
   x = 10/14 = 5/7  

4. Plug x = 5/7 back into y = 4x − 1:  
   y = 4·(5/7) − 1  
     = 20/7 − 7/7  
     = 13/7  

Solution:  
x = 5/7,  y = 13/7.

🤔 User: Is the solution correct? Explain.
🧠 Reasoning Response: Yes. You can check by substituting x=5/7 and y=13/7 into both original equations:

1. Check 2x + 3y = 7  
   Left side = 2·(5/7) + 3·(13/7)  
             = 10/7 + 39/7  
             = 49/7  
             = 7  
   Right side = 7  
   → matches.

2. Check 4x − y = 1  
   Left side = 4·(5/7) − (13/7)  
             = 20/7 − 13/7  
             = 7/7  
             = 1  
   Right side = 1  
   → matche

In [12]:
# You can see the reasoning tokens used by the agent
async for message in thread.get_messages():
    print(f"📝 Message from {message.role}: {message.content[:20]}...")
    if message.metadata.get("usage", None):
        print(f"    🔍 Usage: {message.metadata['usage'].model_dump()}")

📝 Message from AuthorRole.USER: Solve: 2x + 3y = 7, ...
📝 Message from AuthorRole.ASSISTANT: Here’s one way to so...
    🔍 Usage: {'prompt_tokens': 55, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'completion_tokens': 714, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}}
📝 Message from AuthorRole.USER: Is the solution corr...
📝 Message from AuthorRole.ASSISTANT: Yes. You can check b...
    🔍 Usage: {'prompt_tokens': 323, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'completion_tokens': 459, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}}


In [13]:
await thread.delete()

## 🎉 Congratulations!

You've successfully completed the Advanced Agent Features tutorial! Here's what you've mastered:

### ✅ Advanced Agent Capabilities:
1. **Streaming Responses** - Real-time response generation with `invoke_stream()`
2. **Intermediate Steps** - Understanding agent processing with `invoke()` and callbacks
3. **Structured Outputs** - Enforcing consistent response formats with Pydantic models
4. **Reasoning Models** - Enhanced reasoning capabilities with specialized models and token tracking

### 🔧 Key Agent Methods:
- **`get_response()`** - Blocking call that returns the final response
- **`invoke()`** - Non-blocking call that shows intermediate steps and function calls
- **`invoke_stream()`** - Real-time streaming of responses as they're generated

### 🚀 What's Next?
Now that you've mastered individual agent capabilities, you're ready to explore multi-agent systems! Continue with the **Semantic Kernel Multi-Agent Orchestration** notebook to learn about:
- Group chat orchestration patterns
- Concurrent and sequential processing
- Handoff systems
- Human-in-the-loop workflows

### 📚 Additional Resources:
- [Agent API Reference](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-api?pivots=programming-language-python)
- [Reasoning Models Documentation](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/reasoning?tabs=python-secure%2Cpy)
- [Plugin Development](https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/)

Great work! You're now ready to build sophisticated AI agents with advanced capabilities. 🚀